Import basic packages

In [3]:
#
import os
import tarfile
import urllib
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import numpy as np

import hashlib

import sklearn
from sklearn.utils import check_array
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

from scipy import sparse

In [4]:
#
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

# Download data

Run next cell to creat a HOUSING_URL for downloading data

In [5]:
#
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

Create a relative system path datasets\housing, and name it as HOUSING_PATH

In [9]:
HOUSING_PATH = os.path.join('datasets', 'housing')

Define a function of fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH). 

This function would:
1. Check the existence of housing_path in system. If not existing, create this path.
2. download the tgz file from housing_url
3. Extract the tgz file to housing_path

In [19]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    # 1
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    
    # 2
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, filename=tgz_path)
    
    # 3
    tgz_file = tarfile.open(tgz_path)
    tgz_file.extractall(housing_path)
    tgz_file.close()
    

Now run fetch_housing_data to extract the csv file

In [21]:
fetch_housing_data()

Now define a function load_housing_data(housing_path=HOUSING_PATH), which would:
1. Load housing.csv
2. Return a dataframe

Run load_housing_data to create a dataframe 'housing'. Show the head

Show the info of housing to see data type, non-null data count, memory usuage, etc

Show the counts of unique values in oceam_proximity

Show the stats of the numerical attributes

Plot the histograms of all numerical attributes

# Create a test set
Here we set aside a test set. We do this before making any decision to avoid data snooping bias

Define a function split_train_test(dataframe, test_ratio) to split a dataframe to train_set and test_set

Now run the function above to split housing at test_ratio=0.2. To ensure same test data in repeated run of the code, set seed at 42.

The function above is NOT perfect. If new data is appended, one instance may change from test set to train set. It is better to use each instance's (row) identifier (like the primary key) to dicide whether or it should go into the test set.

Define a function test_set_check(idenfier, test_ratio, hash_function) to decide whether an instance should go to test set. Hint: int64 and 256

Define a function split_train_test_by_id(dataframe, test_ratio, id_column, hash_function) to split the dataframe to test_set and train_set

Now split housing by the function above at test_ratio = 0.2. You can select a hash_function from hashlib. You also need to add an index colum (0, 1, 2, ...) to housing before the split

However, if the data was re-ordered by someone else, the result from last method would change. In sum, we'd better use the most stable attributes of the dataframe to create the new "identifier column".

Now let's insert a new column named "id" as longitude * 1000 + latitude. Then use this new column to split housing again.

Now let try to use sklearn.model_selection.train_test_split to split housing to train and test sets

Up to now we are using purely random sampling. But it is still possible to get sampling bias. For example, we may get too many instances of higher income. Now let us try stratified sampling, which would:
1. Select a key attribute
2. Divide this key attribute into several categories
3. Randomly sample within each category
By doing this, we ensure a representative sampling for each category of the key attribute.

We assume that the median income is the key attribute. Now draw the histogram of median_income

Now we define a new attribute "incom_cat" to housing: incom_cat = max(5, ceil(housing.median_income))

Now plot the histogram of incomcat

Now use sklearn.model_selection.StratifiedShuffleSplit to split housing to strat_train_set and strat_test_set data. Note that the stratification is regarding the incom_cat attribute

Now compute the proportions of each category of incom_cat in housing and strat_test_set. Check consistence of the proportions.

Now drop the column of incom_cat from strat_train_set and strat_test_set

# Discover and visualize the data to gain insights

Create dataframe housing as a copy of strat_train_set

Create a scatter plot of longitude vs. latitude, and set alpha at 0.1

Now plot the scatter above again, but:
1. Use population/100 as the scatter size
2. Use median_house_value as color
3. Use jet as colormap
4. Set alpha at 0.4
5. Set legend as "Population"
6. Display the color bar

Compute the correlation matrix of housing. Then display the correlation between median_house_value and other attributes in descending order.

Now plot the scatter matrix of top 4 attributes in last question.

Now plot the scatter of median_house_value vs. median_income, the most promising attribute

Now let's try to create some new attributes and add them to housing:
1. rooms_per_househoold;
2. bedrooms_per_room;
3. population_per_household;

Now calculate the correlation between median_house_value and others again. You should find that the new attribute rooms_per_household and bedrooms_per_room has strong correlation with median_house_value.

# Prepare the data for machine learning algorithms

Recreate dataframe housing by droping "median_house_value" from strat_train_set. Create housing_label as the "median_house_value".

Create housing_num by dropping the non-numerical column from housing

Use sklearn.preprocessing.Imputer to fill missing values in housing_num at attribute median. Return the result as housing_tr

Now let us handle the categorical attribute 'ocean_proximity'. We convert it from text to number by sklearn.preprocessing.LabelEncoder

One problem with the last method is that ML algorithms assume that two nearby values are more similar. But it is not the case for this project. A common solution is to create biniary attribute per category (one-hot). Now use sklearn.preprocessing.OneHotEncoder to convert housing_cat_encoded and return the result as housing_cat_1hot

Now let's assembly a tranformation pipeline

First, define your own estimator "CombinedAttributesAdder". This class would:
1. Has a hyperparameter add_bedrooms_per_room with default value True;
2. Has a fit method but do nothing since we don't change the hyperparameter;
3. Has a transform method that 1) apply on housing.values(); 2) add rooms_per_household and population_per_household; 3) If add_bedrooms_per_room is true, also add bedroom_per_room.

Try to use the fit_transform method of CombinedAttributesAdder to tranform housing and return housing_extra_attribs, which is numpy array. Set add_bedrooms_per_room as False.

Scikit-Learn does not take dataframe as input. So it is necessary to define another transformer "DataFrameSelector" to convert housing to numpy array. This transfomer should take a hyperparameter of a list of column names of interest.

Now apply DataFrameSelector to numerical attributes and categorical attributes.

Now define a tranformation pipeline "num_pipeline" of 4 components to transform housing_num:
1. selector: DataFrameSelector
2. imputer: Imputer to fill missing values at attribute mean
3. attribs_adder: CombinedAttributesAdder to add only two attributes
4. std_scaler: StandardScaler

Now apply fit_transform method of num_pipeline to housing

Now define the cat_pipeline of two components:
1. DataFrameSelector
2. CategoricalEncoder with encodeing='onehot-dense'

We can use sklearn.pipeline.FeatureUnion to merge the two pipelines above

Apply this full_pipe to housing the return the result as housing_prepared

# Select and Train a Model

Create and fit a linear regression model "lin_reg" using housing_prepared and housing_label

Now use this model to predict housing.iloc[:5]. Note that you should tranform the data before the prediction. Print out the prediction and the true answers

Now let's measure this regression model's RMSE on the whole training set.

The RMSE is quite big compared to house value. This is an example of under fitting. Now let's a more powerful model of DecesionTreeRegressor. Then re-calculate the RMSE.

It's like the tree model overfits the model. A better way to evaluate the model is to use cross validation. Now use sklearn.model_selection.cross_val_score to compute the 'neg_mean_squared_error' of each itration. 

Now print out
1. The RMSE of each iteration
2. The mean value of RMSE
3. The std of RMSE
You can define a function to do this

Look at the score you can find that the decision tree is overfitting so badly. Now let us try random forest regressor. Display the training RMSE score over all training data. Also display the score, score mean, and score std from cross validation.

Note that random forest is doing better. The training RMSE is much lower than the validation RMSE, so the model is still overfitted.

# Fine-Tune Your Model 

Run the next cell to establish two grid regions.

Use sklearn.model_selection.GridSearchCV to search the best hyper-parameters of random forest regression.

Show the best estimator and best parameters

Print out the cross validation errors in each step of grid search

Show the feature importance of the best model.

Now define a new pipeline including data preparation and random forest model trainin, so that you can also tune the hyper-parameters in data preparation such as whether or not to add the feature of bedrooms_per_room.

Now you can evaluate your best model in test data. Print out the RMSE on test data.